In [1]:
import tensorflow as tf
import tensorflow.keras as keras 
from tensorflow.keras import layers
import numpy as np


In [2]:
#Go pro dataset
!gdown https://drive.google.com/uc?id=1LgbqjulyTsLspqQ7izi3QXGGIWPbzSmd

Downloading...
From: https://drive.google.com/uc?id=1LgbqjulyTsLspqQ7izi3QXGGIWPbzSmd
To: /content/GOPRO_Large.zip
100% 9.54G/9.54G [01:42<00:00, 92.7MB/s]


In [3]:
!unzip GOPRO_Large.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/GOPR0374_11_02/blur_gamma/000615.png  
  inflating: train/GOPR0374_11_02/blur_gamma/000616.png  
  inflating: train/GOPR0374_11_02/blur_gamma/000617.png  
  inflating: train/GOPR0374_11_02/blur_gamma/000618.png  
  inflating: train/GOPR0374_11_02/blur_gamma/000619.png  
  inflating: train/GOPR0374_11_02/blur_gamma/000620.png  
  inflating: train/GOPR0374_11_02/blur_gamma/000621.png  
  inflating: train/GOPR0374_11_02/blur_gamma/000622.png  
  inflating: train/GOPR0374_11_02/blur_gamma/000623.png  
  inflating: train/GOPR0374_11_02/blur_gamma/000624.png  
  inflating: train/GOPR0374_11_02/blur_gamma/000625.png  
  inflating: train/GOPR0374_11_02/blur_gamma/000626.png  
  inflating: train/GOPR0374_11_02/blur_gamma/000627.png  
  inflating: train/GOPR0374_11_02/blur_gamma/000628.png  
  inflating: train/GOPR0374_11_02/blur_gamma/000629.png  
  inflating: train/GOPR0374_11_02/blur_gamma/000630.png  
  inflating: train/GO

In [4]:
epoch=150
batch_size=16
import os
train_folder = './train'
test_folder="./test"

img_size = (128, 128)
sub_folders_train = sorted(os.path.join(train_folder,name) for name in os.listdir(train_folder) if os.path.isdir(os.path.join(train_folder, name)))
sub_folders_test = sorted(os.path.join(test_folder,name) for name in os.listdir(test_folder) if os.path.isdir(os.path.join(test_folder, name)))
print(sub_folders_train)

['./train/GOPR0372_07_00', './train/GOPR0372_07_01', './train/GOPR0374_11_00', './train/GOPR0374_11_01', './train/GOPR0374_11_02', './train/GOPR0374_11_03', './train/GOPR0378_13_00', './train/GOPR0379_11_00', './train/GOPR0380_11_00', './train/GOPR0384_11_01', './train/GOPR0384_11_02', './train/GOPR0384_11_03', './train/GOPR0384_11_04', './train/GOPR0385_11_00', './train/GOPR0386_11_00', './train/GOPR0477_11_00', './train/GOPR0857_11_00', './train/GOPR0868_11_01', './train/GOPR0868_11_02', './train/GOPR0871_11_01', './train/GOPR0881_11_00', './train/GOPR0884_11_00']


In [5]:
sub_folders_train1_x=[s+'/blur' for s in sub_folders_train]
sub_folders_train1_y=[s+'/sharp' for s in sub_folders_train]
sub_folders_train1_final_x=sorted(os.path.join(i,j )for i in sub_folders_train1_x for j in os.listdir(i) )
sub_folders_train1_final_y=sorted(os.path.join(i,j )for i in sub_folders_train1_y for j in os.listdir(i) )

sub_folders_test1_x=[s+'/blur' for s in sub_folders_test]
sub_folders_test1_y=[s+'/sharp' for s in sub_folders_test]
sub_folders_test1_final_x=sorted(os.path.join(i,j )for i in sub_folders_test1_x for j in os.listdir(i) )
sub_folders_test1_final_y=sorted(os.path.join(i,j )for i in sub_folders_test1_y for j in os.listdir(i) )


In [6]:
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.image import load_img


class DataGenerator(keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, img_size, input_img_paths, target_img_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths = input_img_paths
        self.target_img_paths = target_img_paths

    def __len__(self):
        return len(self.target_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_target_img_paths = self.target_img_paths[i : i + self.batch_size]
        x1 = np.zeros((self.batch_size,) + (32,32) + (3,), dtype="float32")
        x2 = np.zeros((self.batch_size,) + (64,64) + (3,), dtype="float32")
        x3 = np.zeros((self.batch_size,) + self.img_size + (3,), dtype="float32")
        for j, path in enumerate(batch_input_img_paths):
            img = load_img(path, target_size=(32,32))
            x1[j] = img
            x1[j] = x1[j]/255

        for j, path in enumerate(batch_input_img_paths):
            img = load_img(path, target_size=(64,64))
            x2[j] = img
            x2[j] = x2[j]/255

        for j, path in enumerate(batch_input_img_paths):
            img = load_img(path, target_size=(128,128))
            x3[j] = img
            x3[j] = x3[j]/255
        y = np.zeros((self.batch_size,) + self.img_size + (3,), dtype="float32")
        for j, path in enumerate(batch_target_img_paths):
            img = load_img(path, target_size=self.img_size)
            y[j] =img
            y[j] = y[j]/255
          
        return (x1,x2,x3), y

import os
train_data_generated= DataGenerator(
    batch_size, img_size, sub_folders_train1_final_x, sub_folders_train1_final_y
)
val_data_generated = DataGenerator(batch_size, img_size, sub_folders_test1_final_x, sub_folders_test1_final_y)

In [7]:
import tensorflow.keras.layers as layers
def resblock( f,filter,model_name,i):
    #img_size=(256,256)
    input=f
    skip = input
    f = tf.keras.layers.Conv2D(filter, (5,5), padding = 'same', name=str(model_name)+'resblock'+str(i))(input)
    f = tf.keras.layers.Activation('relu')(f)
    f = tf.keras.layers.Conv2D(filter, (5,5), padding = 'same')(f)
    f = tf.keras.layers.Concatenate()([f, skip])     
    return f


In [8]:
def get_model(img_size,model_name,check, y_prev):
  inputs = keras.Input(shape=img_size + (3,))
  if check==1:
    inputs = keras.Input(shape=img_size + (3,))
    x=inputs
  elif check==0:
    inputs = keras.Input(shape=img_size + (3,))
    x=inputs
    x=tf.keras.layers.Concatenate()([inputs,y_prev])
  elif check==2:
    inputs = keras.Input(shape=img_size + (3,))
    x=inputs
    x=tf.keras.layers.Concatenate()([inputs,y_prev])

      # First Scale
  x = layers.Conv2D(64, (5,5), padding="same")(x)
  for i in range(10):
      x=resblock(x, 64,model_name, i)
  x = layers.Conv2D(64, (5,5), padding="same")(x)
  
  if check!=2:
    x = layers.UpSampling2D(2,name=model_name+'_Upsampling1_')(x)


  outputs = layers.Conv2D(3, (5,5), padding="same",name=model_name)(x)  
  # Define the model
  
  if check ==1:
    model = keras.Model(inputs, outputs)
    return model,outputs, inputs
  else:
    model = keras.Model([inputs,y_prev], outputs)
    return model,outputs, inputs




In [9]:
import tensorflow as tf
model1,model1_out,model1_in = get_model((32,32),'1',1, 0)
model2,model2_out,model2_in = get_model((64,64),'2',0, model1_out)
model3,model3_out,model3_in = get_model((128,128),'3',2, model2_out)
model=keras.Model([model1_in,model2_in, model3_in],model3_out)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 64)   4864        ['input_2[0][0]']                
                                                                                                  
 1resblock0 (Conv2D)            (None, 32, 32, 64)   102464      ['conv2d[0][0]']                 
                                                                                                  
 activation (Activation)        (None, 32, 32, 64)   0           ['1resblock0[0][0]']             
                                                                                            

In [10]:
checkpoint_path='logs/checkpoint/-{epoch:02d}-{val_accuracy:.2f}.hdf5'

model_checkpoint=tf.keras.callbacks.ModelCheckpoint(
   checkpoint_path,
    monitor="val_loss",
    verbose=0,
    save_best_only=False,
    save_weights_only=False,
    mode="auto",
    save_freq="epoch",
    options=None,

)

In [11]:
def L2_loss(y_true,y_pred):
    los=tf.reduce_mean(tf.square(y_true-y_pred))
    return los

In [12]:
import math
def SSIM(I1, I2):
    # Adapted from: https://github.com/yzcjtr/GeoNet/blob/master/geonet_model.py
    C1 = 0.01 ** 2
    C2 = 0.03 ** 2

    mu_x = tf.nn.avg_pool(I1, ksize = (3,3), strides=(1,1), padding='SAME')
    mu_y = tf.nn.avg_pool(I2, ksize = (3,3), strides=(1,1), padding='SAME')

    sigma_x  = tf.nn.avg_pool(I1 ** 2, ksize = (3,3), strides=(1,1), padding='SAME') - mu_x ** 2
    sigma_y  = tf.nn.avg_pool(I2 ** 2, ksize = (3,3), strides=(1,1), padding='SAME') - mu_y ** 2
    sigma_xy = tf.nn.avg_pool(I1 * I2 , ksize = (3,3), strides=(1,1), padding='SAME') - mu_x * mu_y
    
    SSIM_n = (2 * mu_x * mu_y + C1) * (2 * sigma_xy + C2)
    SSIM_d = (mu_x ** 2 + mu_y ** 2 + C1) * (sigma_x + sigma_y + C2)
    
    SSIM = SSIM_n / SSIM_d
    return SSIM
def PSNR(original, compressed):
    mse = L2_loss(original,compressed)
    if(mse == 0): 
        return 100.0
    max_pixel = 1
    psnr = 20 * tf.experimental.numpy.log10(max_pixel / tf.math.sqrt(mse))
    return psnr

In [14]:
opt =tf.keras.optimizers.legacy.Adam(lr=0.0001, decay=0.01/epoch )
model.compile(loss=L2_loss,optimizer=opt, metrics=['accuracy',PSNR,SSIM])

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
import tensorflow as tf
from datetime import datetime
logdir = "logs/train_data/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(
    train_data_generated,
    epochs=epoch,
    verbose=1, # Suppress chatty output
    # callbacks=[tensorboard_images(model1_out,model2_out,model3_out,model4_out,model5_out,model6_out,model7_out,model8_out,model9_out,model10_out,fin_model_out), tensorboard_callback],
     #callbacks=[tensorboard_images(model1_out,model2_out,model3_out,model4_out,fin_model_out), tensorboard_callback],
        callbacks=[ tensorboard_callback,model_checkpoint],
    validation_data=val_data_generated

    
)

Epoch 1/150
131/131 [==============================] - 615s 4s/step - loss: 0.2937 - accuracy: 0.5606 - PSNR: 18.4866 - SSIM: 0.4839 - val_loss: 0.0075 - val_accuracy: 0.7046 - val_PSNR: 21.4912 - val_SSIM: 0.6065
Epoch 2/150
131/131 [==============================] - 505s 4s/step - loss: 0.0057 - accuracy: 0.7340 - PSNR: 22.6746 - SSIM: 0.6827 - val_loss: 0.0055 - val_accuracy: 0.7382 - val_PSNR: 22.8323 - val_SSIM: 0.7097
Epoch 3/150
131/131 [==============================] - ETA: 0s - loss: 0.0048 - accuracy: 0.7808 - PSNR: 23.6153 - SSIM: 0.7328

In [ ]:
pic_1=np.array(load_img(sub_folders_test1_final_x[0], target_size=(32,32)))
pic_2=np.array(load_img(sub_folders_test1_final_x[0], target_size=(64,64)))
pic_3=np.array(load_img(sub_folders_test1_final_x[0], target_size=(128,128)))
pic_1= pic_1/255
pic_2= pic_2/255
pic_3= pic_3/255

pic_1= np.expand_dims(pic_1, axis=0)
pic_2= np.expand_dims(pic_2, axis=0)
pic_3= np.expand_dims(pic_3, axis=0)

save_1=[]
save_2=[]
save_3=[]
weights=[]

for epoch in range(epochs):
    pic1=model1.predict([pic_1])
    pic2=model2.predict([pic_1, pic_2])
    pic3=model.predict([pic_1, pic_2, pic_3])

    save_1.append(pic_1)
    save_2.append(pic_2)
    save_3.append(pic_3)
    print("\nStart of epoch %d" % (epoch,))
    #model1(x_)
    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_data_generated):

        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        with tf.GradientTape() as tape:

            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            logits = model(x_batch_train, training=True)  # Logits for this minibatch
            model.get_layers
            # Compute the loss value for this minibatch.
            loss_value = loss_fn(y_batch_train, logits)

        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        grads = tape.gradient(loss_value, model.trainable_weights)
        weights.append(model.trainable_weights)
        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %s samples" % ((step + 1) * 64))
    